# AskQE pipeline (Qwen2.5-14B) — Colab walkthrough

Questa notebook riproduce la pipeline AskQE seguendo le istruzioni e i prompt originali del repository:
- **Generazione (QG/QA/backtranslation)** con `Qwen/Qwen2.5-14B-Instruct-AWQ` via vLLM.
- **Valutazione** con le metriche ufficiali (F1/EM/chrF/BLEU) e SBERT (`all-MiniLM-L6-v2`).

Ogni cella è pensata per essere eseguita in ordine su Google Colab con GPU L4 (22.5 GB).

## 0. Prerequisiti
- Se non hai già il repository locale, clonalo in Colab (`git clone https://github.com/dayeonki/askqe`).
- Assicurati che gli artifact QA originali siano presenti in `QA/` se vuoi usare la sola modalità di scoring.
- Per la generazione, il file di input JSONL deve contenere almeno il campo `en` (o `src`) e opzionalmente `tgt` / `backtranslation`.

In [ ]:
%%bash
# Installazioni minime; aggiungi altre dipendenze del repo se necessario
pip install -q vllm transformers sentence-transformers sacrebleu

# Facoltativo: forza la versione di torch compatibile con la GPU di Colab
# pip install -q torch --index-url https://download.pytorch.org/whl/cu121

In [ ]:
from pathlib import Path

REPO_ROOT = Path.cwd()  # cambia se il notebook non è nella root del repo
MODEL_ID = "Qwen/Qwen2.5-14B-Instruct-AWQ"
GPU_MEM_UTIL = 0.85  # tuning per L4 22.5GB
MAX_TOKENS = 1024

INPUT_JSONL = REPO_ROOT / "data" / "processed" / "example.jsonl"  # sostituisci con il tuo file
GEN_OUTPUT = REPO_ROOT / "askqe_generation_output.jsonl"
SCORE_CSV = REPO_ROOT / "askqe_pipeline_baseline_results.csv"

In [ ]:
%%bash
# Esegui la generazione (QG/QA/backtranslation) con Qwen2.5-14B AWQ via vLLM
python askqe_pipeline_baseline_Judge.py \
  --run-generation \
  --input-file "$INPUT_JSONL" \
  --output-file "$GEN_OUTPUT" \
  --model-id "$MODEL_ID" \
  --gpu-mem-util $GPU_MEM_UTIL \
  --max-tokens $MAX_TOKENS

In [ ]:
import json

preview = []
with open(GEN_OUTPUT, "r", encoding="utf-8") as f:
    for i, line in enumerate(f):
        if i >= 2:
            break
        preview.append(json.loads(line))

preview

In [ ]:
%%bash
# Esegui la sola fase di scoring usando gli artifact QA già presenti in QA/
# Cambia --model se vuoi valutare un'altra cartella (es. llama-70b, yi-9b, ecc.)
python askqe_pipeline_baseline_Judge.py \
  --model llama-70b \
  --qa-dir "$REPO_ROOT/QA" \
  --output "$SCORE_CSV"

In [ ]:
import pandas as pd

df = pd.read_csv(SCORE_CSV)
df.head()